## Generate countries dataset

In [1]:
import pandas as pd
import pickle

In [12]:
cities_ds = "../mydatasets_raw/cities.csv"
cities = pd.read_csv(cities_ds, sep=';')
cities = cities[['ASCII Name', 'Country name EN', 'Population']]
cities.columns = ['city', 'country', 'population']
cities = cities[cities['population'] != 0]
cities = cities.drop_duplicates(subset=['city'], keep=False)
#cities = cities[isinstance(cities['city'], str)]
cities = cities[cities.apply(lambda row : isinstance(row['city'], str), axis=1)]
cities = cities[cities.apply(lambda row : isinstance(row['country'], str), axis=1)]
print(len(cities))
cities.head()
#df['is_correct'] = df.apply(lambda row : row['filtered_answer'] in country_names[row['correct_country']], axis=1) 

110826


,city,country,population
0,Huayllati,Peru,515
1,Duraznopampa,Peru,249
2,Kimbiri,Peru,4369
3,Urb. Santo Domingo,Peru,5000
6,Kimbe,Papua New Guinea,18847


In [ ]:
selection = cities.sample(10000)

In [ ]:
cities_questions_augmented = pd.DataFrame(columns=["statement", "label", "city", "country", "correct_country", "question", "question_with_answer"])
for index, row in selection.iterrows():
    row = {"statement": "The city of " + row['city'] + " is in " + row['country'] + ".", 
            "label": 1, 
            "city": row['city'], 
            "country": row['country'], 
            "correct_country": row['country'], 
            "question": "In which country is the city of " + row['city'] + " located?", 
            "question_with_answer": "In which country is the city of " + row['city'] + " located? " + row['country'] + "."}
    row_df = pd.Series(row).to_frame().T
    cities_questions_augmented = pd.concat([cities_questions_augmented, row_df], axis=0, ignore_index=True)

In [ ]:
cities_questions_augmented

In [ ]:
cities_questions_augmented.to_csv("../../datasets/cities_expansions/cities_questions_augmented_2.csv")

## Create countries dictionary

In [13]:
set(cities['country'])
country_names = {}
for country in set(cities['country']):
    country_names[country] = [country]
country_names['Russian Federation'].append('Russia')
country_names['Viet Nam'].append('Vietnam')
country_names['Venezuela, Bolivarian Rep. of'].append('Venezuela')
country_names['United States'].append('USA')
country_names['United States'].append('the USA')
country_names['United States'].append('The USA')
country_names['United States'].append('The United States')
country_names['Congo, Democratic Republic of the'].append('Congo')
country_names['Hong Kong, China'].append('Hong Kong')
country_names['Iran, Islamic Rep. of'].append('Iran')
country_names['West Bank and Gaza Strip'].append('Palestine')
country_names['United Kingdom'].append('UK')
country_names['United Kingdom'].append('the UK')
country_names['United Kingdom'].append('The UK')
country_names["Korea, Dem. People's Rep. of"].append('North Korea')
country_names['Korea, Republic of'].append('South Korea')
country_names['Sudan, The Republic of'].append('Sudan')

In [16]:
with open('../mydatasets/cities_dictoinary.pkl', 'wb') as f:
    pickle.dump(country_names, f)

## Create 'statement' dataset

In [19]:
cities = pd.read_csv("../../datasets/cities_expansions/cities_questions_augmented_2.csv")

In [20]:
cities.head()

,Unnamed: 0,statement,label,city,country,correct_country,question,question_with_answer
0,0,The city of Kostomloty Pierwsze is in Poland.,1,Kostomloty Pierwsze,Poland,Poland,In which country is the city of Kostomloty Pie...,In which country is the city of Kostomloty Pie...
1,1,The city of Bognankro is in Côte d'Ivoire.,1,Bognankro,Côte d'Ivoire,Côte d'Ivoire,In which country is the city of Bognankro loca...,In which country is the city of Bognankro loca...
2,2,The city of Le Raincy is in France.,1,Le Raincy,France,France,In which country is the city of Le Raincy loca...,In which country is the city of Le Raincy loca...
3,3,The city of Tobol is in Kazakhstan.,1,Tobol,Kazakhstan,Kazakhstan,In which country is the city of Tobol located?,In which country is the city of Tobol located?...
4,4,The city of Tayabas Ibaba is in Philippines.,1,Tayabas Ibaba,Philippines,Philippines,In which country is the city of Tayabas Ibaba ...,In which country is the city of Tayabas Ibaba ...


In [21]:
len(cities)

9984

In [22]:
cities_new = cities[['question']]

In [23]:
cities_new.columns = ['statement']

In [24]:
cities_new.to_csv("../../datasets/cities_questions_augmented_2_for_activations.csv")

Then run: 
python3 generate_acts.py --model_family Llama3 --model_size 8B --model_type chat --layers 12 --datasets cities_questions_augmented_2_for_activations --device cuda:0 --output_dir acts/


In [43]:
cities = pd.read_csv("../mydatasets/cities_labeled.csv")
cities.head()

,question,answer,filtered_answer,correct_answer,is_correct
0,In which country is the city of Kostomloty Pie...,Poland. I finished,Poland,Poland,True
1,In which country is the city of Bognankro loca...,I don't know. I finished,I don't know,Côte d'Ivoire,False
2,In which country is the city of Le Raincy loca...,France. I finished,France,France,True
3,In which country is the city of Tobol located?,Kazakhstan. I finished,Kazakhstan,Kazakhstan,True
4,In which country is the city of Tayabas Ibaba ...,Philippines. I finished,Philippines,Philippines,True


In [28]:
cities = cities[["question", "answer", "filtered_answer", "correct_country", "is_correct"]]
cities = cities.rename(columns={"correct_country": "correct_answer"})
cities.head()

,question,answer,filtered_answer,correct_answer,is_correct
0,In which country is the city of Kostomloty Pie...,Poland. I finished,Poland,Poland,True
1,In which country is the city of Bognankro loca...,I don't know. I finished,I don't know,Côte d'Ivoire,False
2,In which country is the city of Le Raincy loca...,France. I finished,France,France,True
3,In which country is the city of Tobol located?,Kazakhstan. I finished,Kazakhstan,Kazakhstan,True
4,In which country is the city of Tayabas Ibaba ...,Philippines. I finished,Philippines,Philippines,True


In [49]:
cities["answer"] = ['-' for _ in range(len(cities))]
cities["filtered_answer"] = ['-' for _ in range(len(cities))]
cities["is_correct"] = ['-' for _ in range(len(cities))]

In [50]:
cities.head()

,question,answer,filtered_answer,correct_answer,is_correct
0,In which country is the city of Kostomloty Pie...,-,-,Poland,-
1,In which country is the city of Bognankro loca...,-,-,Côte d'Ivoire,-
2,In which country is the city of Le Raincy loca...,-,-,France,-
3,In which country is the city of Tobol located?,-,-,Kazakhstan,-
4,In which country is the city of Tayabas Ibaba ...,-,-,Philippines,-


In [51]:
cities.to_csv("../mydatasets/cities.csv", index=False)

## Create birth years dataset

In [38]:
births_raw = pd.read_csv("../mydatasets_raw/birth_years.csv")
births_raw = births_raw[births_raw.apply(lambda row : isinstance(row['firstname'], str), axis=1)]
births_raw = births_raw[births_raw.apply(lambda row : isinstance(row['lastname'], str), axis=1)]
births_raw = births_raw[births_raw.apply(lambda row : isinstance(row['birthDate'], str), axis=1)]
births_raw.head()

,lastname,firstname,articleNum,birthDate,birthMonth,birthDay,zodiac
1,Aaron,Hank,46,1934-02-05,2.0,5.0,Aquarius
2,Abacha,Sani,2,1943-09-20,9.0,20.0,Virgo
3,Abbado,Claudio,9,1933-06-26,6.0,26.0,Cancer
4,Abbas,Mahmoud,306,1935-03-26,3.0,26.0,Aries
5,Abdel Rahman,Omar,21,1938-05-03,5.0,3.0,Taurus


In [39]:
birth_years = pd.DataFrame(columns=["question", "answer", "filtered_answer", "correct_answer", "is_correct"])
for index, row in births_raw.iterrows():
    new_row = {"question": "What year was " + row['firstname'] + " " + row['lastname'] + " born?", 
            "answer": '-', 
            "filtered_answer": '-', 
            "correct_answer": row['birthDate'][:4], 
            "is_correct": '-'}
    row_df = pd.Series(new_row).to_frame().T
    birth_years = pd.concat([birth_years, row_df], axis=0, ignore_index=True)

In [40]:
birth_years.head()

,question,answer,filtered_answer,correct_answer,is_correct
0,What year was Hank Aaron born?,-,-,1934,-
1,What year was Sani Abacha born?,-,-,1943,-
2,What year was Claudio Abbado born?,-,-,1933,-
3,What year was Mahmoud Abbas born?,-,-,1935,-
4,What year was Omar Abdel Rahman born?,-,-,1938,-


In [42]:
birth_years.to_csv("../mydatasets/birth_years.csv", index=False)

## Create football leagues dataset

In [55]:
football_raw = pd.read_csv("../mydatasets_raw/football_leagues.csv")
football_raw.head()

,competition,season,rank,squad,games,wins,draws,losses,goals_for,goals_against,...,cards_red,shots_on_target_against,saves,clean_sheets,shots_on_target,games_starts,games_complete,games_subs,unused_subs,points_per_match
0,Premier League,2010-2011,1,Manchester Utd,38,23,11,4,78,37,...,3.0,139,102,15,218,418,NaN,99,NaN,2.11
1,Premier League,2010-2011,2,Chelsea,38,21,8,9,69,33,...,1.0,148,115,15,241,418,NaN,107,NaN,1.87
2,Premier League,2010-2011,3,Manchester City,38,21,8,9,60,33,...,5.0,153,120,18,182,418,NaN,102,NaN,1.87
3,Premier League,2010-2011,4,Arsenal,38,19,11,8,72,43,...,6.0,150,106,13,243,418,NaN,107,NaN,1.79
4,Premier League,2010-2011,5,Tottenham,38,16,14,8,55,46,...,2.0,183,137,8,202,418,NaN,96,NaN,1.63


In [60]:
football = pd.DataFrame(columns=["question", "answer", "filtered_answer", "correct_answer", "is_correct"])
for index, row in football_raw.iterrows():
    new_row = {"question": "Which team finished in position " + str(row['rank']) + " in the " + row['season'] + " " + row["competition"] + "?", 
            "answer": '-', 
            "filtered_answer": '-', 
            "correct_answer": row['squad'], 
            "is_correct": '-'}
    row_df = pd.Series(new_row).to_frame().T
    football = pd.concat([football, row_df], axis=0, ignore_index=True)

In [64]:
football.head()

,question,answer,filtered_answer,correct_answer,is_correct
0,Which team finished in position 1 in the 2010-...,-,-,Manchester Utd,-
1,Which team finished in position 2 in the 2010-...,-,-,Chelsea,-
2,Which team finished in position 3 in the 2010-...,-,-,Manchester City,-
3,Which team finished in position 4 in the 2010-...,-,-,Arsenal,-
4,Which team finished in position 5 in the 2010-...,-,-,Tottenham,-


In [65]:
football.to_csv("../mydatasets/football_leagues.csv", index=False)

## Create medals dataset

In [69]:
medals_raw = pd.read_csv("../mydatasets_raw/olympic_medals.csv")
print(len(medals_raw))
medals_raw = medals_raw[medals_raw.apply(lambda row : isinstance(row['Medal'], str), axis=1)]
print(len(medals_raw))
medals_raw = medals_raw[medals_raw.apply(lambda row : row['Season'] == 'Summer', axis=1)]
print(len(medals_raw))
medals_raw = medals_raw[medals_raw.apply(lambda row : row['Medal'] == 'Gold', axis=1)]
print(len(medals_raw))
medals_raw.head()

271116
39783
34088
11459


,ID,Name,Sex,Age,Height,Weight,Team,NOC,Games,Year,Season,City,Sport,Event,Medal
3,4,Edgar Lindenau Aabye,M,34.0,NaN,NaN,Denmark/Sweden,DEN,1900 Summer,1900,Summer,Paris,Tug-Of-War,Tug-Of-War Men's Tug-Of-War,Gold
42,17,Paavo Johannes Aaltonen,M,28.0,175.0,64.0,Finland,FIN,1948 Summer,1948,Summer,London,Gymnastics,Gymnastics Men's Team All-Around,Gold
44,17,Paavo Johannes Aaltonen,M,28.0,175.0,64.0,Finland,FIN,1948 Summer,1948,Summer,London,Gymnastics,Gymnastics Men's Horse Vault,Gold
48,17,Paavo Johannes Aaltonen,M,28.0,175.0,64.0,Finland,FIN,1948 Summer,1948,Summer,London,Gymnastics,Gymnastics Men's Pommelled Horse,Gold
79,21,Ragnhild Margrethe Aamodt,F,27.0,163.0,NaN,Norway,NOR,2008 Summer,2008,Summer,Beijing,Handball,Handball Women's Handball,Gold


In [75]:
medals = pd.DataFrame(columns=["question", "answer", "filtered_answer", "correct_answer", "is_correct"])
for index, row in medals_raw.iterrows():
    new_row = {"question": "Which country won gold in " + row['Event'] + " in the " + row["Games"] + " Olympics?", 
            "answer": '-', 
            "filtered_answer": '-', 
            "correct_answer": row['Team'], 
            "is_correct": '-'}
    row_df = pd.Series(new_row).to_frame().T
    medals = pd.concat([medals, row_df], axis=0, ignore_index=True)

In [77]:
medals.head()

,question,answer,filtered_answer,correct_answer,is_correct
0,Which country won gold in Tug-Of-War Men's Tug...,-,-,Denmark/Sweden,-
1,Which country won gold in Gymnastics Men's Tea...,-,-,Finland,-
2,Which country won gold in Gymnastics Men's Hor...,-,-,Finland,-
3,Which country won gold in Gymnastics Men's Pom...,-,-,Finland,-
4,Which country won gold in Handball Women's Han...,-,-,Norway,-


In [79]:
medals.to_csv("../mydatasets/medals.csv", index=False)